In [1]:
from keras.layers import *
from keras.models import *
from keras.backend import sum as Sum
from keras.backend import ones
import keras

Using TensorFlow backend.


In [2]:
featurelen=2048
dictlen=8495
sentencelen=39
EMBEDDING_DIM=512
visibletime=1

cnn_X = 12
cnn_Y = 12
filters = 768



In [3]:
def get_model():
    word_inp = Input((sentencelen,))
    emd_word = Embedding(dictlen, 256)(word_inp)
    emd_word = BatchNormalization()(emd_word)
    
    img_inp = Input((filters,cnn_X*cnn_Y))
    img_inp = RepeatVector(sentencelen)(img_inp)

    
    att_inp = Input((sentencelen, cnn_X*cnn_Y)) # Probabilities
    
    att_inp = TimeDistributed(RepeatVector(filters))(att_inp)
    
    z_vec = TimeDistributed(Multiply())([img_inp, att_inp])
    
    z_t = TimeDistributed(sum(keep_dims=False))[z_vec]

    
    
    
    return model

In [4]:
word_inp = Input((sentencelen,))
emd_word = Embedding(dictlen, 256)(word_inp)
emd_word = BatchNormalization()(emd_word)

img_in = Input((filters,cnn_X*cnn_Y))
img_inp = Flatten()(img_in)
img_inp = RepeatVector(sentencelen)(img_inp)
img_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(img_inp)


att_in = Input((sentencelen, cnn_X*cnn_Y)) # Probabilities
att_inp = Flatten()(att_in)
att_inp = RepeatVector(filters)(att_inp)
att_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(att_inp)

# mult  = multiply([img_inp, att_inp])
# # z = Sum(mult, axis=3)
# print mult
# mult=Permute((1,3,2),input_shape=(39,768,144))(mult)
# print mult
# z=TimeDistributed(AveragePooling1D(pool_size=144))(mult)
# z=TimeDistributed(BatchNormalization())(z)
# print z
# print emd_word
# z=Reshape((sentencelen,filters))(z)
# lstm_in = concatenate([z, emd_word])
# lstm_out = LSTM(1536,return_sequences=True,dropout=0.5)(lstm_in)
# model = Model(inputs=[word_inp, img_in, att_in],outputs=lstm_out)
# ini_ones=ones((39,144,))
# print ini_ones
# print model.summary()
# # IMG_INP = Input((filters,cnn_X*cnn_Y))
# # WORD_INP = Input((sentencelen,))
# # ini_ones = ones((cnn_X*cnn_Y,))
# # LSTM_OUT = model([WORD_INP, IMG_INP, ini_ones ])
# # attend = TimeDistributed(Dense(cnn_X*cnn_Y,activation="softmax"))(LSTM_OUT)
# # LSTM_OUT = model([WORD_INP, IMG_INP, attend])
# # fin = TimeDistributed(Dense(dictlen, activation="softmax"))(LSTM_OUT) 

# # fin_model = Model(inputs=[IMG_INP, WORD_INP], outputs=fin)
# # fin_model.summary()

In [12]:
word_inp = Input((sentencelen,))
emd_word = Embedding(dictlen, 256)(word_inp)
emd_word = BatchNormalization()(emd_word)

img_in = Input((filters,cnn_X*cnn_Y))
img_inp = Flatten()(img_in)
img_inp = RepeatVector(sentencelen)(img_inp)
img_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(img_inp)


att_in = Input((sentencelen, cnn_X*cnn_Y)) # Probabilities
att_inp = Flatten()(att_in)
att_inp = RepeatVector(filters)(att_inp)
att_inp = Reshape((sentencelen,filters,cnn_X*cnn_Y))(att_inp)

mult=multiply([img_inp,att_inp])
mult=Permute((1,3,2),input_shape=(39,768,144))(mult)
z=TimeDistributed(AveragePooling1D(pool_size=144))(mult)
z=TimeDistributed(BatchNormalization())(z)
z=Reshape((sentencelen,filters))(z)
lstm_in = concatenate([z, emd_word])
lstm_out = LSTM(1536,return_sequences=True,dropout=0.5)(lstm_in)
model = Model(inputs=[word_inp, img_in, att_in],outputs=lstm_out)
print mult
print model.summary()
# print att_inp.input

# ini_ones=ones((39,144,128))
IMG_INP = Input((filters,cnn_X*cnn_Y))
WORD_INP = Input((sentencelen,))
# ini_ones = np.ones((128,sentencelen,cnn_X*cnn_Y))
# print ini_ones
ones_i=Input((cnn_X*cnn_Y,))
ones=RepeatVector(sentencelen)(ones_i)
LSTM_OUT = model([WORD_INP, IMG_INP,ones ])
attend = TimeDistributed(Dense(cnn_X*cnn_Y,activation="softmax"))(LSTM_OUT)
LSTM_OUT = model([WORD_INP, IMG_INP, attend])
fin = TimeDistributed(Dense(dictlen, activation="softmax"))(LSTM_OUT) 

fin_model = Model(inputs=[IMG_INP, WORD_INP, ones_i], outputs=fin)
fin_model.summary()

Tensor("permute_7/transpose:0", shape=(?, 39, 144, 768), dtype=float32)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_23 (InputLayer)            (None, 768, 144)      0                                            
____________________________________________________________________________________________________
input_24 (InputLayer)            (None, 39, 144)       0                                            
____________________________________________________________________________________________________
flatten_15 (Flatten)             (None, 110592)        0           input_23[0][0]                   
____________________________________________________________________________________________________
flatten_16 (Flatten)             (None, 5616)          0           input_24[0][0]                   
___________________

In [9]:
def get_model_with_attention(cnn_feature_size, vocab_size, max_token_len, embedding_dim = 512):
	#model here
	cnn_X=cnn_feature_size[0]
	cnn_Y=cnn_feature_size[1]
	filters=cnn_feature_size[2]
	
	emd_word=Sequential()
	emd_word.add(Embedding(vocab_size,embedding_dim,input_length=max_token_len))
	emd_word.add(BatchNormalization())

	img_inp=Sequential()
	img_inp.add(Flatten(input_shape=(filters,cnn_X*cnn_Y)))
	img_inp.add(RepeatVector(max_token_len))
	img_inp.add(Reshape((max_token_len,filters,cnn_X*cnn_Y)))

	att_inp=Sequential()
	att_inp.add(Flatten(input_shape=(max_token_len,cnn_X*cnn_Y)))
	att_inp.add(RepeatVector(filters))
	att_inp.add(Reshape((max_token_len,filters,cnn_X*cnn_Y)))

	mult=multiply([img_inp.output,att_inp.output])
	mult=Permute((1,3,2),input_shape=(max_token_len,filters,cnn_X*cnn_Y))(mult)
	z=TimeDistributed(AveragePooling1D(pool_size=cnn_X*cnn_Y))(mult)
	z=TimeDistributed(BatchNormalization())(z)
	z=Reshape((max_token_len,filters))(z)
	lstm_in = concatenate([z, emd_word.output])
	lstm_out = LSTM(1536,return_sequences=True,dropout=0.5)(lstm_in)
	model = Model(inputs=[emd_word.input, img_inp.input, att_inp.input],outputs=lstm_out)

	IMG_INP = Input((filters,cnn_X*cnn_Y))
	WORD_INP = Input((max_token_len,))

	ones_i=Input((cnn_X*cnn_Y,))
	ones=RepeatVector(max_token_len)(ones_i)
	LSTM_OUT = model([WORD_INP, IMG_INP,ones ])
	attend = TimeDistributed(Dense(cnn_X*cnn_Y,activation="softmax"))(LSTM_OUT)
	LSTM_OUT = model([WORD_INP, IMG_INP, attend])
	fin = TimeDistributed(Dense(vocab_size, activation="softmax"))(LSTM_OUT) 

	fin_model = Model(inputs=[IMG_INP, WORD_INP, ones_i], outputs=fin)
	return fin_model

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_15 (InputLayer)            (None, 144)           0                                            
____________________________________________________________________________________________________
input_14 (InputLayer)            (None, 50)            0                                            
____________________________________________________________________________________________________
input_13 (InputLayer)            (None, 768, 144)      0                                            
____________________________________________________________________________________________________
repeat_vector_15 (RepeatVector)  (None, 50, 144)       0           input_15[0][0]                   
___________________________________________________________________________________________